In [1]:
using IPGBs
using JuMP
using LinearAlgebra
using Random

┌ Warning: Circular dependency detected. Precompilation will be skipped for:
│   Base.PkgId(Base.UUID("63c18a36-062a-441e-b654-da1e3ab1ce7c"), "KernelAbstractions")
│   Base.PkgId(Base.UUID("eae2faf6-b232-58cb-a410-7764fda2830c"), "ChainRulesCoreExt")
│   Base.PkgId(Base.UUID("1285c0f1-ff9f-5867-b66e-0f359bcf09ba"), "SpecialFunctionsExt")
│   Base.PkgId(Base.UUID("052768ef-5323-5732-b1bb-66c8b64840ba"), "CUDA")
│   Base.PkgId(Base.UUID("85068d23-b5fb-53f1-8204-05c2aba6942f"), "SparseArraysExt")
│   Base.PkgId(Base.UUID("13011619-4c7c-5ef0-948f-5fc81565cd05"), "AtomixCUDAExt")
│   Base.PkgId(Base.UUID("66d79d19-2cc4-5b0b-ac7a-b340256d1ecd"), "LinearAlgebraExt")
│   Base.PkgId(Base.UUID("02a925ec-e4fe-4b08-9a7e-0d78e3d38ccd"), "cuDNN")
└ @ Base.Precompilation precompilation.jl:511
[ Info: Precompiling IPGBs [cd26b91b-75b4-4f6e-a355-d2a0d136006d] (cache misses: include_dependency fsize change (2), mismatched flags (16))


In [2]:
Random.seed!(0)
n = 5
c = rand(1:10, 1, n)
A = rand(1:10, 1, n)
b = [round(Int, sum(A) / 2)]

1-element Vector{Int64}:
 8

In [3]:
gr = IPGBs.FourTi2.graver(A, [true, true, true, true, true]);
gr

51×5 Matrix{Int64}:
 1   0   0   0  -2
 0   1   0   0  -2
 0   0   1   0  -8
 0   0   0   1  -3
 1  -1   0   0   0
 1   0   0  -1   1
 1   0  -1   0   6
 0   1   0  -1   1
 0   1  -1   0   6
 0   0   1  -1  -5
 1   1   0  -1  -1
 1   0  -1   1   3
 1   1  -1   0   4
 ⋮              
 0   4  -1   0   0
 1   2   0  -2   0
 2   1   0  -2   0
 0   3   0  -2   0
 3   0   0  -2   0
 1   1   1  -4   0
 0   2   1  -4   0
 2   0   1  -4   0
 0   0   2  -5  -1
 0   1   2  -6   0
 1   0   2  -6   0
 0   0   3  -8   0

In [4]:
function lawrence_lifting(A)
    m, n = size(A)
    I5 = I(5)
    Z5 = zeros(Int, m, n) 
    return [A Z5; I5 I5]
end

lawrence_lifting (generic function with 1 method)

In [5]:
gr_lift = IPGBs.FourTi2.graver(lawrence_lifting(A), [true for _ in 1:10])
gr_lift

51×10 Matrix{Int64}:
 0   1   0   0  -2   0  -1   0   0   2
 0   0   1   0  -8   0   0  -1   0   8
 0   0   0   1  -3   0   0   0  -1   3
 1   0   0   0  -2  -1   0   0   0   2
 1  -1   0   0   0  -1   1   0   0   0
 1   0   0  -1   1  -1   0   0   1  -1
 1   0  -1   0   6  -1   0   1   0  -6
 0   1   0  -1   1   0  -1   0   1  -1
 0   1  -1   0   6   0  -1   1   0  -6
 0   0   1  -1  -5   0   0  -1   1   5
 1   1   0  -1  -1  -1  -1   0   1   1
 1   0  -1   1   3  -1   0   1  -1  -3
 1   1  -1   0   4  -1  -1   1   0  -4
 ⋮                   ⋮              
 0   4  -1   0   0   0  -4   1   0   0
 1   2   0  -2   0  -1  -2   0   2   0
 2   1   0  -2   0  -2  -1   0   2   0
 0   3   0  -2   0   0  -3   0   2   0
 3   0   0  -2   0  -3   0   0   2   0
 1   1   1  -4   0  -1  -1  -1   4   0
 0   2   1  -4   0   0  -2  -1   4   0
 2   0   1  -4   0  -2   0  -1   4   0
 0   0   2  -5  -1   0   0  -2   5   1
 0   1   2  -6   0   0  -1  -2   6   0
 1   0   2  -6   0  -1   0  -2   6   0
 0   0

In [6]:
new_A = lawrence_lifting(A)
new_b = [10000, 10000, 10000, 10000, 10000, 10000]
new_c = [1 1 1 1 1 1 1 1 1 1]
gb_ipgbs = groebner_basis(new_A, new_b, new_c; apply_normalization=false, truncation_type=:None);

┌ Info: P&L bounded case
│   i = 10
└   length(pl_state.markov) = 4


In [7]:
G = IPGBs.FourTi2.groebner(new_A, new_c)
gb_4ti2 = [G[i, :] for i in 1:size(G, 1)];

In [13]:
@show IPGBs.GBTools.isincluded(gb_ipgbs, gb_4ti2)
@show IPGBs.GBTools.isincluded(gb_4ti2, gb_ipgbs);

IPGBs.GBTools.isincluded(gb_ipgbs, gb_4ti2) = true
IPGBs.GBTools.isincluded(gb_4ti2, gb_ipgbs) = true
